# Part 2: Viewshed case study

In the second part we will demonstrate the use of GRASS for a small viewshed case study.
The goal is to compute and analyze the area a driver would see from a road.

Topics covered:
 * Python scripting
 * manipulating vector data ([v.build.polylines](https://grass.osgeo.org/grass-stable/manuals/v.build.polylines.html), [v.to.points](https://grass.osgeo.org/grass-stable/manuals/v.to.points.html))
 * vector attributes ([v.db.select](https://grass.osgeo.org/grass-stable/manuals/v.db.select.html))
 * viewshed computation ([r.viewshed](https://grass.osgeo.org/grass-stable/manuals/r.viewshed.html))
 * simple parallelization ([multiprocessing.Pool](https://docs.python.org/3/library/multiprocessing.html))
 * region handling ([grass.script.region_env](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region_env))
 * raster algebra ([r.mapcalc](https://grass.osgeo.org/grass-stable/manuals/r.mapcalc.html))
 * reprojecting ([r.proj](https://grass.osgeo.org/grass-stable/manuals/r.proj.html))
 * resampling ([r.resample.interp](https://grass.osgeo.org/grass-stable/manuals/r.resample.interp.html))
 * raster mask ([r.mask](https://grass.osgeo.org/grass-stable/manuals/r.mask.html))
 * raster as numpy array ([grass.script.array](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#module-script.array))

In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
from tqdm import tqdm

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init(os.path.expanduser("~/grassdata"), "dix_park", "PERMANENT")

### Data preparation
We will first derive viewpoints along the road *Umstead Drive* (vector `umstead_drive_segments`) that we extracted in the first part of the workshop.

1. Because the road consists of several segments, we will merge them into one.
2. Create new vector of points along the line with distance 50 m.

In [ ]:
gs.run_command("v.build.polylines", input="umstead_drive_segments", output="umstead_drive", cats="first")
gs.run_command("v.to.points", input="umstead_drive", type="line", output="viewpoints", dmax=50)

We want to compute the visibility using DSM, however some points may fall on top of a tree, so we need to filter those out.

3. Compute height above ground (DSM - DTM).
4. Extract height above ground for the viewpoint locations.

In [ ]:
gs.mapcalc("diff = dsm - ground")
gs.run_command("v.what.rast", map="viewpoints", layer=2, raster="diff", column="height")
gs.run_command("r.colors", map="diff", color="differences")

See the newly computed attribute data. This example shows how the attribute data can be loaded into pandas:

In [ ]:
import json
import pandas as pd
pd.DataFrame(json.loads(gs.read_command("v.db.select", map="viewpoints", columns="cat,height", layer=2, format="json"))["records"])

Visualize the viewpoints with the height-above-ground raster. You can filter the points based on the height:

In [ ]:
img = gj.GrassRenderer()
img.d_rast(map="diff")
img.d_vect(map="umstead_drive")
img.d_vect(map="viewpoints", layer=2, where="height < 2", size=10, icon="basic/pin")
img.d_legend(raster="diff")
img.show()

### Viewshed computation
To get the cumulative viewshed, we will compute viewsheds from all the viewpoints we generated earlier.
First, we get the list coordinates of the viewpoints that are likely lying on the ground:

In [ ]:
viewpoints = gs.read_command('v.out.ascii', input='viewpoints',
                             separator='comma', layer=2, where="height < 2").strip().splitlines()
viewpoints = [p.split(",") for p in viewpoints]
viewpoints

We will now compute the viewshed from each viewpoint in a loop. We set max distance of 300 m. Each viewshed will be named `viewshed_cat`.

In [ ]:
%%time
maps = []
for x, y, cat in tqdm(viewpoints):
    name = f"viewshed_{cat}"
    gs.run_command("r.viewshed", input="dsm", output=name,
                   coordinates=(x, y), max_distance=300)
    maps.append(name)

Since these are independent runs, we can easily parallelize the r.viewshed calls using Python multiprocessing.
We define a function that computes the viewshed and returns the name of the output or None in case of error:

In [ ]:
%%time
from grass.exceptions import CalledModuleError
from multiprocessing import Pool, cpu_count


def viewshed(point):
    x, y, cat = point
    x, y = float(x), float(y)
    name = f"viewshed_{cat}"
    try:
        gs.run_command("r.viewshed", input="dsm", output=name,
                       coordinates=(x, y), max_distance=300)
        return f"viewshed_{cat}"
    except CalledModuleError:
        return None

# run with the number of CPUs available
# proc = cpu_count()
proc = 1
with Pool(processes=proc) as pool:
    maps = pool.map_async(viewshed, viewpoints).get()
print(maps)

One trick to speedup viewshed computation is to limit the computation only
to the actual area given by the maxdistance option. To do that we will locally modify the computational region
and pass the environment to the module directly. The current computational region won't be affected.

In [ ]:
%%time
from grass.exceptions import CalledModuleError
from multiprocessing import Pool, cpu_count


def viewshed(point):
    x, y, cat = point
    x, y = float(x), float(y)
    max_distance = 300
    # copy current environment
    env = os.environ.copy()
    # set GRASS_REGION variable using region_env function
    env["GRASS_REGION"] = gs.region_env(align="dsm",
                                        e=x + max_distance,
                                        w=x - max_distance,
                                        n=y + max_distance,
                                        s=y - max_distance)
    name = f"viewshed_{cat}"
    try:
        gs.run_command("r.viewshed", input="dsm", output=name,
                      coordinates=(x, y), max_distance=max_distance, env=env)
        return f"viewshed_{cat}"
    except CalledModuleError:
        return None

# run with the number of CPUs available
# proc = cpu_count()
proc = 1
with Pool(processes=proc) as pool:
    maps = pool.map_async(viewshed, viewpoints).get()
print(maps)
print(f"Viewshed num cells: {gs.raster_info(maps[0])['cells']}")
print(f"DSM num cells: {gs.raster_info('dsm')['cells']}")


### Cumulative viewshed
Finally, we can compute the cumulative viewshed, which aggregates viewsheds from multiple viewpoints. In this way you can e.g., identify the most frequently visible areas from the road.

First, let's check we have the viewshed rasters ready:

In [ ]:
gs.list_grouped(type="raster", pattern="viewshed_*")["PERMANENT"]

Since by default viewshed rasters have no data in areas that are not visible, we will use r.series method *count*:

In [ ]:
# cumulative viewshed
gs.run_command("r.series", input=maps, output="cumulative_viewshed", method="count")
# set color of cumulative viewshed from grey to yellow
color_rules = """0% 70:70:70
100% yellow"""
gs.write_command("r.colors", map="cumulative_viewshed", rules="-", stdin=color_rules)

Let's visualize the results in an interactive map:

In [ ]:
fig = gj.InteractiveMap(width=600)
# Add vector and layer control to map
fig.add_raster("cumulative_viewshed", opacity=0.6)
fig.add_vector("umstead_drive")
fig.add_layer_control(position="bottomright")
fig.show()

And create a 3D rendering with draped cumulative viewshed over the DSM:

In [ ]:
img = gj.Grass3dRenderer()
img.render(elevation_map="dsm", resolution_fine=1, color_map="cumulative_viewshed",
           vline="umstead_drive", vline_width=3, vline_color="white",
           position=[0.5, 0.8], height=2500, perspective=15)
img.overlay.d_legend(raster="cumulative_viewshed", at=(60, 97, 87, 92), color="white", flags="f")
img.show()

## Data reprojection and resampling
Next, we will analyze the cumulative viewshed to see how much greenery a driver would see on the way. To do that we compute NDVI:

1. We first reproject R and NIR Landsat bands from NCSPM sample dataset we already have available. Module r.proj respects the current region (extent and resolution), but you can set resolution to certain value, we use 28.5 m which is the original resolution.
2. We resample each band to our current resolution (1m) using bilinear interpolation. Note that we don't have to do that if nearest neighbor resampling is sufficient.


In [ ]:
for band in [30, 40]:
    gs.run_command("r.proj", location="nc_spm_08_grass7", mapset="PERMANENT", input=f"lsat7_2002_{band}", method="nearest", resolution=28.5)
    gs.run_command("r.resamp.interp", input=f"lsat7_2002_{band}", output=f"lsat7_2002_{band}_1m", method="bilinear")

Compute NDVI:

In [ ]:
gs.run_command("i.vi", viname="ndvi", red="lsat7_2002_30_1m", nir="lsat7_2002_40_1m", output="ndvi")

In [ ]:
img = gj.GrassRenderer()
img.d_rast(map="ndvi")
img.d_legend(raster="ndvi")
img.show()

## Mask
Now let's analyze what is the distribution of NDVI within the visible area. We will mask the data by the visible area:

![Mask](img/mask.gif)

In [ ]:
gs.run_command("r.mask", raster="cumulative_viewshed", maskcats="1 thru 6")
data = gs.parse_command("r.univar", map="ndvi", flags="g")
print(f"Average NDVI of visible cells: {data['mean']} ± {data['stddev']}")

In [ ]:
img = gj.GrassRenderer()
img.d_rast(map="ndvi")
img.d_legend(raster="ndvi", flags="d")
img.show()

Let's see the histogram of visible NDVI using d.histogram:

In [ ]:
img = gj.GrassRenderer()
img.d_histogram(map="ndvi", bgcolor="grey")
img.show()

### Read as numpy array
It is also easy to use the results as a numpy array and then use other Python libraries to analyze the data:

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from grass.script import array as garray

In [ ]:
ndvi = garray.array(mapname="ndvi", null='nan')
ndvi

In [ ]:
sns.set_style('darkgrid')
sns.histplot(ndvi.ravel(), kde=True)

Finally, remove the mask:

In [ ]:
gs.run_command("r.mask", flags="r")